In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv
/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip
/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip


In [ ]:
!unzip /kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip
!unzip /kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip

In [3]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from time import time as timer
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import random_split, Dataset, DataLoader

In [4]:
train_dir = "/kaggle/working/train"

train_imgs = sorted(os.listdir(train_dir))
print(train_imgs[:10])

img1 = Image.open(train_dir + "/" + train_imgs[0])
img1.size

['cat.0.jpg', 'cat.1.jpg', 'cat.10.jpg', 'cat.100.jpg', 'cat.1000.jpg', 'cat.10000.jpg', 'cat.10001.jpg', 'cat.10002.jpg', 'cat.10003.jpg', 'cat.10004.jpg']


(500, 374)

In [5]:
class CustomDataset(Dataset):
    def __init__(self, images_dir, transform=None):
        self.transforms = transform
        self.imgs_dir = images_dir
        self.arr = sorted(os.listdir(self.imgs_dir))

    def __len__(self):
        return len(os.listdir(self.imgs_dir))
    
    def __getitem__(self, idx):
        img_name = self.arr[idx]
        label = self.arr[idx][:3]
        if label == 'cat':
            label = 0
        else:
            label = 1

        img_path = os.path.join(self.imgs_dir, img_name)

        img = Image.open(img_path).convert("RGB")

        if self.transforms:
            img = self.transforms(img)
        
        return img, label

In [72]:
train_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

test_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [73]:
train_dataset = CustomDataset(
    images_dir = "/kaggle/working/train", transform = train_transform
)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [75]:
import torch.optim as optim
from torchvision import transforms
from torchvision.models import resnet50, ResNet50_Weights

# Загружаем ResNet50 V2 с предобученными весами
weights = ResNet50_Weights.IMAGENET1K_V2  # именно V2-версия
model = resnet50(weights=weights)

# Заменяем последний полносвязный слой под нашу задачу
num_classes = 2
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, num_classes)

# Если хотите, можно зафиксировать (заморозить) все слои кроме последнего:
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Критерий и оптимизатор
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Обучение
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")

# Оценка точности на последней эпохе
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"Accuracy: {correct}/{total} = {correct/total:.4f}")

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 188MB/s]
Epoch 1/10: 100%|██████████| 782/782 [01:16<00:00, 10.21it/s]


Epoch 1, Loss: 0.1407


Epoch 2/10: 100%|██████████| 782/782 [01:16<00:00, 10.23it/s]


Epoch 2, Loss: 0.1007


Epoch 3/10: 100%|██████████| 782/782 [01:16<00:00, 10.24it/s]


...

Epoch 7, Loss: 0.0835


Epoch 8/10: 100%|██████████| 782/782 [01:16<00:00, 10.20it/s]


Epoch 8, Loss: 0.0832


Epoch 9/10:   6%|▌         | 46/782 [00:04<01:13, 10.00it/s]


KeyboardInterrupt: 

In [76]:
test_dir = "/kaggle/working/test"

test_imgs = sorted(os.listdir(test_dir), key=lambda x: int(x.split('.')[0]))
print(test_imgs[:10])

img1 = Image.open(test_dir + "/" + test_imgs[5])
img1.size

['1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg', '10.jpg']


(499, 375)

In [77]:
class CustomTestDataset(Dataset):
    def __init__(self, images_dir, transform=None):
        self.transforms = transform
        self.imgs_dir = images_dir
        self.arr = sorted(os.listdir(self.imgs_dir), key=lambda x: int(x.split('.')[0]))

    def __len__(self):
        return len(os.listdir(self.imgs_dir))
    
    def __getitem__(self, idx):
        img_name = self.arr[idx]

        img_path = os.path.join(self.imgs_dir, img_name)

        img = Image.open(img_path).convert("RGB")

        if self.transforms:
            img = self.transforms(img)
        
        return img, idx

In [78]:
test_dataset = CustomTestDataset("/kaggle/working/test", test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [79]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [80]:
all_preds = []
img_idxs = []

with torch.no_grad():
    for images, idxs in tqdm(test_loader):
        images = images.to(device)
        
        outputs = model(images)
        p = outputs.argmax(1)

        all_preds.extend(p.cpu().numpy())
        img_idxs.extend(idxs)

100%|██████████| 391/391 [00:37<00:00, 10.46it/s]


In [81]:
all_preds[-100:]

[1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0]

In [82]:
subm = pd.read_csv('/kaggle/input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv')
subm

,id,label
0,1,0.5
1,2,0.5
2,3,0.5
3,4,0.5
4,5,0.5
...,...,...
12495,12496,0.5
12496,12497,0.5
12497,12498,0.5
12498,12499,0.5


In [83]:
subm['label'] = all_preds

In [84]:
subm

,id,label
0,1,1
1,2,1
2,3,1
3,4,1
4,5,0
...,...,...
12495,12496,0
12496,12497,0
12497,12498,1
12498,12499,1


In [85]:
subm.to_csv('submfinal.csv', index=False)